# Kanawha / FFRD Metadata Demo

In [162]:
import rdflib

In [163]:
g = rdflib.Graph()
g.parse("./kanawha.ttl", format="turtle")

<Graph identifier=N768e00367e6244a79fc5d467918b0b9e (<class 'rdflib.graph.Graph'>)>

## Query for models based a model creator name
Find models created by a particlar person.

In [164]:
model_creator = "Mark McBroom"

In [165]:
query_creator = f"""
PREFIX rascat: <http://www.example.org/rascat/0.1#>
PREFIX dcterms: <http://purl.org/dc/terms/>
SELECT DISTINCT ?title ?description ?model
WHERE {{
    ?model a rascat:RasModel .
    ?model dcterms:title ?title .
    ?model dcterms:description ?description .
    ?model dcterms:creator ?creators .
    ?model dcterms:creator [foaf:name "{model_creator}"] .
}}
"""

In [166]:
results = g.query(query_creator)
for row in results:
    title = row[0]
    description = row[1]
    model = row[2]
    print(title, model)

GauleySummersville_BLE_FEMA http://example.ffrd.fema.gov/kanawha/models/GSummersville_B.prj
GauleySummersville_BLE-C_FEMA http://example.ffrd.fema.gov/kanawha/models/GSummersville_C.prj
GauleyLower_BLE-C_FEMA http://example.ffrd.fema.gov/kanawha/models/GauleyLower_BLE_FEM.prj


## Query for models based on 2D mesh cell count
Find models with more than X cells.

In [167]:
cell_count = 400000

In [168]:
query_cell_count = f"""
PREFIX rascat: <http://www.example.org/rascat/0.1#>
PREFIX dcterms: <http://purl.org/dc/terms/>
SELECT ?title ?description ?model ?geometry ?cellCount
WHERE {{
    ?model a rascat:RasModel .
    ?model dcterms:title ?title .
    ?model dcterms:description ?description .
    ?model rascat:hasGeometry ?geometry .
    ?geometry rascat:hasMesh2D ?mesh2D .
    ?mesh2D rascat:cellCount ?cellCount .
    FILTER (?cellCount > {cell_count})
}}
ORDER BY DESC(?cellCount)    
"""

In [169]:
results = g.query(query_cell_count)
for row in results:
    title = row[0]
    description = row[1]
    model = row[2]
    geometry = row[3]
    cell_count = row[4]
    print(title, cell_count, geometry)

GauleyLower_BLE-C_FEMA 1088388 http://example.ffrd.fema.gov/kanawha/models/GauleyLower_BLE_FEM.g01
UpperNew_Upper 1064723 http://example.ffrd.fema.gov/kanawha/models/UpperNew_Upper.g01
GauleySummersville_BLE-C_FEMA 954995 http://example.ffrd.fema.gov/kanawha/models/GSummersville_C.g01
UpperNew_Lower 800701 http://example.ffrd.fema.gov/kanawha/models/UpperNew_Lower.g03
ElkMiddle 761035 http://example.ffrd.fema.gov/kanawha/models/ElkMiddle.g01
ElkMiddle 761035 http://example.ffrd.fema.gov/kanawha/models/ElkMiddle.g02
GauleySummersville_BLE_FEMA 587360 http://example.ffrd.fema.gov/kanawha/models/GSummersville_B.g01
Upper Kanawha 537254 http://example.ffrd.fema.gov/kanawha/models/UpperKanawha.g01
New-Little River 517580 http://example.ffrd.fema.gov/kanawha/models/New-LittleRiver.g01
Bluestone Local - Compass 2D BLE 417852 http://example.ffrd.fema.gov/kanawha/models/BluestoneLocal.g01
Watershed G3 411065 http://example.ffrd.fema.gov/kanawha/models/WatershedG3.g01
Watershed G3 411065 http://

## Query based on calibration metrics
Identify the top 10 Plan calibration hydrographs by according to the Nash-Sutcliffe Efficiency metric.

In [170]:
limit = 10

In [171]:
query_calibration = f"""
PREFIX rascat: <http://www.example.org/rascat/0.1#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX usgs_gages: <https://waterdata.usgs.gov/monitoring-location/>
SELECT ?title ?description ?model ?flow ?gage ?nse ?flowTitle ?gageTitle ?hydroType ?plan ?planTitle
WHERE {{
    ?model a rascat:RasModel .
    ?model dcterms:title ?title .
    ?model dcterms:description ?description .
    ?model rascat:hasPlan ?plan .
    ?plan rascat:hasUnsteadyFlow ?flow .
    ?plan rascat:hasCalibrationHydrograph ?hydro .
    ?plan dcterms:title ?planTitle .
    ?flow dcterms:title ?flowTitle .
    ?hydro rascat:fromStreamgage ?gage .
    ?hydro rascat:hydrographType ?hydroType .
    ?gage dcterms:identifier ?gageID .
    ?gage dcterms:title ?gageTitle .
    ?hydro rascat:nse ?nse .
    FILTER (?hydroType = "Flow")
}}
ORDER BY ASC(?nse)
LIMIT {limit}
"""

In [172]:
results = g.query(query_calibration)
for row in results:
    title = row[0]
    description = row[1]
    model = row[2]
    flow = row[3]
    gage = row[4]
    nse = row[5]
    flow_title = row[6]
    gage_title = row[7]
    hydro_type = row[8]
    plan = row[9]
    plan_title = row[10]
    print(title, nse, plan_title, plan, gage)

UpperNew_Upper -7.27 Jun2016 http://example.ffrd.fema.gov/kanawha/models/UpperNew_Upper.p04 https://waterdata.usgs.gov/monitoring-location/03164000
CoalRiver -1.97 Jan1996 http://example.ffrd.fema.gov/kanawha/models/CoalRiver.p03 https://waterdata.usgs.gov/monitoring-location/03198500
WatershedG9 -1.5141 Jan-1995 Calibration http://example.ffrd.fema.gov/kanawha/models/WatershedG9.p05 https://waterdata.usgs.gov/monitoring-location/03184000
UpperNew_Upper -1.22 Jun2016 http://example.ffrd.fema.gov/kanawha/models/UpperNew_Upper.p04 https://waterdata.usgs.gov/monitoring-location/03161000
UpperNew_Lower -0.343 Jun2016 http://example.ffrd.fema.gov/kanawha/models/UpperNew_Lower.p07 https://waterdata.usgs.gov/monitoring-location/03168000
UpperNew_Upper -0.34 Jan1996 http://example.ffrd.fema.gov/kanawha/models/UpperNew_Upper.p03 https://waterdata.usgs.gov/monitoring-location/03164000
Kanawha_G1 -0.17704 Jan-1995 Calibration http://example.ffrd.fema.gov/kanawha/models/Kanawha_G1.p05 https://wate

## List all surface roughness landuse/landcover sources

In [188]:
query_lulc = """
PREFIX rascat: <http://www.example.org/rascat/0.1#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX usgs_gages: <https://waterdata.usgs.gov/monitoring-location/>
SELECT ?landuseDesc (GROUP_CONCAT(DISTINCT ?title; separator=", ") as ?titles)
WHERE {
    ?model a rascat:RasModel .
    ?model dcterms:title ?title .
    ?model dcterms:description ?description .
    ?model rascat:hasGeometry ?geom .
    ?geom rascat:hasRoughness ?rough .
    ?rough rascat:hasLanduseLandcover ?landuse .
    ?landuse dcterms:description ?landuseDesc .
}
GROUP BY ?landuseDesc
"""

In [190]:
results = g.query(query_lulc)
for row in results:
    lulc_desc = row[0]
    titles = row[1]
    print(lulc_desc, titles)

Custom land cover analysis of NAIP 2022 imagery BasinG6, G5, Greenbrier_G7, Greenbrier_G8, Kanawha_G1, Kanawha_G2, Watershed G3, Watershed G4, WatershedG9
National Land Cover Database 2019 (CONUS) Bluestone Local - Compass 2D BLE, Bluestone_Upper, CoalRiver, GauleySummersville_BLE_FEMA, New-Little River, Upper Kanawha, UpperNew_Lower, UpperNew_Upper
Remote Sensing Data processed by WSP ElkMiddle
National Agriculture Imagery Program (NAIP) imagery processed using machine learning tools. Pulled August 2022 GauleySummersville_BLE-C_FEMA, GauleyLower_BLE-C_FEMA


## Identify any models that used a certain USGS gage for calibration

In [175]:
gage = "03187500"

In [176]:
query_gage = f"""
PREFIX rascat: <http://www.example.org/rascat/0.1#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX usgs_gages: <https://waterdata.usgs.gov/monitoring-location/>
SELECT DISTINCT ?model ?gage ?gageID
WHERE {{
    ?model a rascat:RasModel .
    ?model rascat:hasPlan ?plan .
    ?plan rascat:hasCalibrationHydrograph ?hydro .
    ?hydro rascat:fromStreamgage ?gage .
    ?gage dcterms:identifier ?gageID .
    FILTER (?gageID = "{gage}")
}}
"""

In [177]:
results = g.query(query_gage)
for row in results:
    model = row[0]
    print(model)

http://example.ffrd.fema.gov/kanawha/models/GSummersville_B.prj
http://example.ffrd.fema.gov/kanawha/models/GSummersville_C.prj


## Map USGS gages associated with a model

In [178]:
model = "New-LittleRiver.prj"

In [179]:
query_gages = f"""
PREFIX rascat: <http://www.example.org/rascat/0.1#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX usgs_gages: <https://waterdata.usgs.gov/monitoring-location/>
PREFIX kanawha: <http://ffrd.fema.gov/models/kanawha/>
SELECT DISTINCT ?model ?gage ?gageID
WHERE {{
    ?model a rascat:RasModel .
    ?model rascat:hasPlan ?plan .
    ?plan rascat:hasCalibrationHydrograph ?hydro .
    ?hydro rascat:fromStreamgage ?gage .
    ?gage dcterms:identifier ?gageID .
    FILTER (?model = kanawha:{model})
}}
"""

In [180]:
results = g.query(query_gages)
gage_urls = []
for row in results:
    gage = row[1]
    gage_urls.append(gage)
    gageID = row[2]
    print(gage, gageID)

In [181]:
gage_urls

[]

In [182]:
import json

import requests
from bs4 import BeautifulSoup

def get_ld_json(url: str) -> dict:
    parser = "html.parser"
    req = requests.get(url)
    soup = BeautifulSoup(req.text, parser)
    return json.loads("".join(soup.find("script", {"type":"application/ld+json"}).contents))

In [183]:
gage_ld_jsons = []
for gage_url in gage_urls:
    gage_ld_jsons.append(get_ld_json(gage_url))

In [184]:
print(gage_ld_jsons)

[]


In [185]:
from ipyleaflet import Map, Marker
from ipywidgets import HTML
import statistics

center_lat = statistics.mean([float(i['geo']['latitude']) for i in gage_ld_jsons])
center_lng = statistics.mean([float(i['geo']['longitude']) for i in gage_ld_jsons])

center = (center_lat, center_lng)

m = Map(center=center, zoom=9)

for j in gage_ld_jsons:
    location = (float(j['geo']['latitude']), float(j['geo']['longitude']))
    name = j['name']
    description = j['description']
    gage_url = j['@id']
    gage_popup = HTML()
    gage_popup.value = f"<h3>{name}</h3><p>Used to calibrate <b>{model}</b></p><p>{description}</p><p><a target=\"_blank\" rel=\"noopener noreferrer\" href=\"{gage_url}\">{gage_url}</a></p>"
    marker = Marker(location=location, draggable=False, title=name)
    marker.popup = gage_popup
    m.add_layer(marker)
m

StatisticsError: mean requires at least one data point